In [18]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Problem 2

In [19]:
train_raw = pd.read_csv("train.csv")

# Ignoring excluded columns
cols_to_drop = ["id", "date", "zipcode", "Unnamed: 0"]
train_df = train_raw.drop(columns=cols_to_drop, errors="ignore")

# Splitting features and target
X_train = train_df.drop(columns=["price"])
y_train = train_df["price"] / 1000

# Scaling features (fit on training only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Training model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predicting on training data
y_train_pred = model.predict(X_train_scaled)

# Computing training metrics
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print("Training Set Metrics")
print("MSE:", mse_train)
print("R^2:", r2_train)

# Printing coefficients
print("\nLinear Regression Coefficients:")
for feature, coef in zip(X_train.columns, model.coef_):
    print(feature, ":", coef)

print("\nIntercept:", model.intercept_)

Training Set Metrics
MSE: 31486.167775794882
R^2: 0.7265334318706018

Linear Regression Coefficients:
bedrooms : -12.52196186860659
bathrooms : 18.527632513048715
sqft_living : 56.748836801162035
sqft_lot : 10.881868446111161
floors : 8.043720836862803
waterfront : 63.7428995598697
view : 48.20010852419161
condition : 12.964269364046077
grade : 92.23147482243314
sqft_above : 48.29008886178811
sqft_basement : 27.137032467668593
yr_built : -67.64311741342638
yr_renovated : 17.27137953034478
lat : 78.37573693207808
long : -1.0352030837350685
sqft_living15 : 45.57765781263847
sqft_lot15 : -12.930090977794954

Intercept: 520.414834000001


In [20]:
# Reading test data
test_raw = pd.read_csv("test.csv")

# Ignorning excluded columns
test_df = test_raw.drop(columns=cols_to_drop, errors="ignore")

# Splitting features and target
X_test = test_df.drop(columns=["price"])
y_test = test_df["price"] / 1000

# Scaling test features
X_test_scaled = scaler.transform(X_test)

# Predicting on test data
y_test_pred = model.predict(X_test_scaled)

# Computing testing metrics
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("\nTesting Set Metrics")
print("MSE:", mse_test)
print("R^2:", r2_test)


Testing Set Metrics
MSE: 57628.15470567037
R^2: 0.6543560876120955


### Which features contribute mostly to the linear regression model?
Since all of the features are standardized, the coefficients are directly comparable across features. The features that contribute the most to the linear regression model include grade (92.2), lat (78.4), year_built(-67.6), waterfront (63.7), and sqft_living (56.7). All of these features make sense as the quality of the house has a major impact on price, location and the year the house was built plays a significant role in determining house value, and waterfront view and sqft_living strongly affect the house price.
### Is the model fitting the data well?
The training R-squared (0.73) indicates that about 73% of the variance in house prices is explained by the model, and the testing R-squared (0.65) shows that the model explains 65% of price variation. The model fits reasonably well for a simple linear regression, but there is room for improvement.
### How large is the model error?
There is some overfitting in the model, but not extreme overfitting, because the R-squared only drops from 0.73 to 0.65. The model fits the training data better than the testing data. The model generalizes well, but is still off by a substantial amount for individual prediction error.
### How do the training and testing MSE relate?
The testing MSE is higher than the training MSE, which is expected because the model was optimized on the training data. The increase means some loss of performance on unseen data, which means mild overfitting. However, since the difference is not extreme and the R-squared remains relatively high on the test set, the model still performs reasonably well.

# Problem 3

In [23]:
# Converting scaled training data to numpy
X_train_np = X_train_scaled
y_train_np = y_train.to_numpy(dtype=float)

X_test_np = X_test_scaled
y_test_np = y_test.to_numpy(dtype=float)

# Adding bias term
X_train_b = np.c_[np.ones((X_train_np.shape[0], 1)), X_train_np]
X_test_b  = np.c_[np.ones((X_test_np.shape[0], 1)), X_test_np]

# Closed-form solution
theta = np.linalg.pinv(X_train_b) @ y_train_np

# Function to predict a testing point
def predict_point(x_new, theta):
    x_new = np.array(x_new, dtype=float).reshape(1, -1)
    x_new_b = np.c_[np.ones((1, 1)), x_new]
    return float(x_new_b @ theta)

# Predicting values
y_train_pred_cf = X_train_b @ theta
y_test_pred_cf  = X_test_b @ theta

# Calculating and printing metrics
mse_train_cf = mean_squared_error(y_train_np, y_train_pred_cf)
r2_train_cf  = r2_score(y_train_np, y_train_pred_cf)

mse_test_cf = mean_squared_error(y_test_np, y_test_pred_cf)
r2_test_cf  = r2_score(y_test_np, y_test_pred_cf)

print("\nTraining Set Metrics")
print("MSE:", mse_train_cf)
print("R^2:", r2_train_cf)

print("\nTesting Set Metrics")
print("MSE:", mse_test_cf)
print("R^2:", r2_test_cf)

# Print intercept and coefficients
print("\nClosed-Form Intercept:", theta[0])

print("\nClosed-Form Coefficients:")
for feature, coef in zip(X_train.columns, theta[1:]):
    print(feature, ":", coef)


Training Set Metrics
MSE: 31486.16777579488
R^2: 0.7265334318706018

Testing Set Metrics
MSE: 57628.1547056704
R^2: 0.6543560876120953

Closed-Form Intercept: 520.4148340000008

Closed-Form Coefficients:
bedrooms : -12.521961868606303
bathrooms : 18.52763251304865
sqft_living : 56.748836801162014
sqft_lot : 10.881868446111227
floors : 8.04372083686274
waterfront : 63.74289955986932
view : 48.20010852419148
condition : 12.964269364046025
grade : 92.23147482243324
sqft_above : 48.29008886178805
sqft_basement : 27.137032467668448
yr_built : -67.64311741342641
yr_renovated : 17.271379530344895
lat : 78.37573693207827
long : -1.0352030837349417
sqft_living15 : 45.577657812638385
sqft_lot15 : -12.930090977794892


In [24]:
print("\nComparison with sklearn LinearRegression")

print("\nTraining Set")
print("Sklearn MSE:", mse_train)
print("Closed-Form MSE:", mse_train_cf)

print("Sklearn R^2:", r2_train)
print("Closed-Form R^2:", r2_train_cf)

print("\nTesting Set")
print("Sklearn MSE:", mse_test)
print("Closed-Form MSE:", mse_test_cf)

print("Sklearn R^2:", r2_test)
print("Closed-Form R^2:", r2_test_cf)


Comparison with sklearn LinearRegression

Training Set
Sklearn MSE: 31486.167775794882
Closed-Form MSE: 31486.16777579488
Sklearn R^2: 0.7265334318706018
Closed-Form R^2: 0.7265334318706018

Testing Set
Sklearn MSE: 57628.15470567037
Closed-Form MSE: 57628.1547056704
Sklearn R^2: 0.6543560876120955
Closed-Form R^2: 0.6543560876120953


The closed-form model produces the same MSE and R² values as the sklearn LinearRegression model on both the training and testing sets. This confirms that the closed-form solution correctly uses the ordinary least squares regression formula. Since sklearn’s LinearRegression also has the same output, the implementation is equivalent to the package solution.

# Problem 4

In [25]:
# Building polynomial features
def make_poly_features(X_np, p):
    X_np = np.array(X_np, dtype=float)
    
    # Creating list
    features = [X_np ** deg for deg in range(1, p + 1)]
    
    # Concatenating column-wise
    return np.concatenate(features, axis=1)

# Fitting polynomial regression with closed-form solution
def fit_poly_regression(X_train_np, y_train_np, p):
    
    # Building polynomial de/sign matrix
    X_poly = make_poly_features(X_train_np, p)
    
    # Adding bias column of ones
    X_b = np.c_[np.ones((X_poly.shape[0], 1)), X_poly]
    
    # Closed-form least squares solution
    theta = np.linalg.pinv(X_b) @ y_train_np
    
    return theta

# Predicting using learned theta
def predict_poly(X_np, theta, p):
    
    # Building polynomial features
    X_poly = make_poly_features(X_np, p)
    
    # Adding bias
    X_b = np.c_[np.ones((X_poly.shape[0], 1)), X_poly]
    
    # Returning predictions
    return X_b @ theta

# Evaluating model
def evaluate_poly_model(X_train_np, y_train_np, X_test_np, y_test_np, p):
    
    # Fitting model
    theta = fit_poly_regression(X_train_np, y_train_np, p)
    
    # Predictions
    y_train_pred = predict_poly(X_train_np, theta, p)
    y_test_pred  = predict_poly(X_test_np, theta, p)
    
    # Metrics
    mse_train = mean_squared_error(y_train_np, y_train_pred)
    r2_train  = r2_score(y_train_np, y_train_pred)
    
    mse_test = mean_squared_error(y_test_np, y_test_pred)
    r2_test  = r2_score(y_test_np, y_test_pred)
    
    return theta, mse_train, r2_train, mse_test, r2_test

In [26]:
feature = "sqft_living"
col_idx = list(X_train.columns).index(feature)

# Keeping X as (N, 1)
X_train_np = np.array(X_train_scaled[:, [col_idx]], dtype=float)
X_test_np  = np.array(X_test_scaled[:,  [col_idx]], dtype=float)

# y as numpy
y_train_np = y_train.to_numpy(dtype=float)
y_test_np  = y_test.to_numpy(dtype=float)

# Training models for p <= 5 and calculating metrics
results = []

for p in range(1, 6):
    theta, mse_tr, r2_tr, mse_te, r2_te = evaluate_poly_model(
        X_train_np, y_train_np, X_test_np, y_test_np, p
    )
    
    results.append({
        "p": p,
        "MSE_train": mse_tr,
        "R2_train": r2_tr,
        "MSE_test": mse_te,
        "R2_test": r2_te
    })

results_df = pd.DataFrame(results)

# Printing results
print("\nPolynomial Regression Results using X = sqft_living (p <= 5)")
print(results_df.to_string(index=False))


Polynomial Regression Results using X = sqft_living (p <= 5)
 p    MSE_train  R2_train      MSE_test   R2_test
 1 57947.526161  0.496709  88575.978543  0.468736
 2 54822.665116  0.523849  71791.679479  0.569406
 3 53785.194716  0.532860  99833.483763  0.401216
 4 52795.774758  0.541453 250979.274285 -0.505331
 5 52626.111955  0.542927 570616.914821 -2.422464


As the polynomial degree increases, the training MSE gets smaller and the training R-squared gets larger. This happens because more complex models fit the training data more closely. On the testing data, performance improves from degree 1 to degree 2. The test MSE decreases and the test R-squared increases, meaning the model predicts better. However, after degree 2, the test MSE increases sharply and the test R-squared decreases, becoming negative for higher degrees. This shows that higher-degree models are overfitting. They fit the training data well but perform poorly on new data.

# Problem 5

In [27]:
# Core gradient descent function
def gd_train_linear_regression(X, y, alpha, num_iters, theta0=None, return_history=False):
    
    m, n = X.shape

    # initializing theta
    theta = np.zeros(n) if theta0 is None else theta0.copy()

    cost_history = [] if return_history else None

    for _ in range(num_iters):

        # predicting
        preds = X @ theta

        # errors
        errors = preds - y

        # gradient
        gradient = (2.0 / m) * (X.T @ errors)

        # parameter update
        theta = theta - alpha * gradient

    if return_history:
        return theta, cost_history

    return theta


# Evaluation function
def evaluate_theta(X_train_b, y_train, X_test_b, y_test, theta):

    y_train_pred = X_train_b @ theta
    y_test_pred  = X_test_b  @ theta

    metrics = {
        "mse_train": mean_squared_error(y_train, y_train_pred),
        "r2_train":  r2_score(y_train, y_train_pred),
        "mse_test":  mean_squared_error(y_test, y_test_pred),
        "r2_test":   r2_score(y_test, y_test_pred)
    }

    return metrics


# Wrapper function
def train_and_evaluate_gd(X_train_b, y_train, X_test_b, y_test, alpha, num_iters, theta0=None):

    theta = gd_train_linear_regression(
        X_train_b,
        y_train,
        alpha=alpha,
        num_iters=num_iters,
        theta0=theta0
    )

    metrics = evaluate_theta(X_train_b, y_train, X_test_b, y_test, theta)

    return theta, metrics

In [29]:
alphas = [0.01, 0.1, 0.5]
iters_list = [10, 50, 100]

rows = []

for alpha in alphas:
    for num_iters in iters_list:

        # Training model
        theta = gd_train_linear_regression(
            X_train_b,
            y_train_np,
            alpha=alpha,
            num_iters=num_iters
        )

        # Evaluating metrics
        metrics = evaluate_theta(
            X_train_b, y_train_np,
            X_test_b,  y_test_np,
            theta
        )

        # Printing theta (separating theta0)
        np.set_printoptions(suppress=True, precision=4)

        theta0 = theta[0]
        first_five = theta[1:6]   # first 5 coefficients after intercept

        print("Learning Rate (alpha):", alpha)
        print("Iterations:", num_iters)
        print(f"Theta0 (Intercept): {theta0:.4f}")
        print("First 5 Thetas:", np.round(first_five, 4))
        print("\n")

        # Storing metrics for table
        rows.append([
            alpha,
            num_iters,
            metrics["mse_train"],
            metrics["r2_train"],
            metrics["mse_test"],
            metrics["r2_test"]
        ])

# Creating results table
results_df = pd.DataFrame(
    rows,
    columns=[
        "alpha",
        "num_iters",
        "mse_train",
        "r2_train",
        "mse_test",
        "r2_test"
    ]
)

print("Metrics Table")
clean_df = results_df.copy()

# Formatting numeric columns
for col in clean_df.columns:
    if col not in ["alpha", "num_iters"]:
        clean_df[col] = clean_df[col].map(lambda x: f"{x:.4e}")

print(clean_df.to_string(index=False))

Learning Rate (alpha): 0.01
Iterations: 10
Theta0 (Intercept): 95.1980
First 5 Thetas: [11.9286 20.2833 32.1165  5.3808  9.5255]


Learning Rate (alpha): 0.01
Iterations: 50
Theta0 (Intercept): 330.8955
First 5 Thetas: [ 6.0053 23.8214 54.6659  3.6375 11.122 ]


Learning Rate (alpha): 0.01
Iterations: 100
Theta0 (Intercept): 451.3976
First 5 Thetas: [-3.6569 19.2418 56.955   2.6937 10.4482]


Learning Rate (alpha): 0.1
Iterations: 10
Theta0 (Intercept): 464.5357
First 5 Thetas: [-4.6456 18.7334 57.1255  2.53   10.5525]


Learning Rate (alpha): 0.1
Iterations: 50
Theta0 (Intercept): 520.4074
First 5 Thetas: [-12.4572  17.6212  57.1878   7.8533   8.006 ]


Learning Rate (alpha): 0.1
Iterations: 100
Theta0 (Intercept): 520.4148
First 5 Thetas: [-12.5191  18.4242  56.7696  10.2707   8.061 ]


Learning Rate (alpha): 0.5
Iterations: 10
Theta0 (Intercept): 520.4148
First 5 Thetas: [-40545159.6361 -60243904.6471 -66812541.2293 -25606793.6371
 -37406765.8138]


Learning Rate (alpha): 0.5
Iterat

For α = 0.01, the algorithm improves as the number of iterations increases. The MSE decreases and R² increases from negative values to positive values by 100 iterations. For α = 0.1, the algorithm converges much faster and achieves the best performance. By 50 iterations, both training and testing R-squared values are already around 0.7 and stabilize by 100 iterations. The MSE also plateaus, showing that the algorithm reached the optimal solution within about 50–100 iterations. For α = 0.5, the algorithm diverges. MSE becomes extremely large, and R-squared becomes highly negative. This shows that the learning rate is too large. In conclusion, the algorithm converges when the learning rate is appropriately chosen (a = 0.1), requires around 50–100 iterations to stabilize, and fails to converge when the learning rate is too large.

# Problem 6

In [30]:
def gd_train_ridge_regression(X, y, alpha, num_iters, lam, theta0=None):
    m, n = X.shape
    theta = np.zeros(n) if theta0 is None else theta0.astype(float).copy()

    for _ in range(num_iters):
        preds = X @ theta
        errors = preds - y

        # Fitting the gradient
        gradient = (2.0 / m) * (X.T @ errors)

        # Ridge gradient scaled by m
        ridge_term = (2.0 * lam / m) * theta
        ridge_term[0] = 0.0

        gradient += ridge_term

        theta = theta - alpha * gradient

    return theta

In [33]:
np.random.seed(42)

# Simulating data
N = 1000
X = np.random.uniform(-2, 2, size=N)

# e_i ~ N(0, 2)  => std = sqrt(2)
e = np.random.normal(loc=0.0, scale=np.sqrt(2), size=N)

y = 1 + 2 * X + e

# Designing matrix with intercept
X_b = np.column_stack([np.ones(N), X])

# Iterating
num_iters = 5000

# Preventing lamba from exploding
def pick_alpha(lam):
    if lam <= 10:
        return 0.1
    elif lam <= 100:
        return 0.01
    else:
        return 0.001

# Fitting linear + ridge models and printing
lambdas = [0, 1, 10, 100, 1000, 10000]

for lam in lambdas:
    alpha = pick_alpha(lam)

    if lam == 0:
        theta = gd_train_linear_regression(X_b, y, alpha=alpha, num_iters=num_iters)
        model_name = "Linear (lambda=0)"
    else:
        theta = gd_train_ridge_regression(X_b, y, alpha=alpha, num_iters=num_iters, lam=lam)
        model_name = f"Ridge (lambda={lam})"

    # Predictions on full dataset
    y_pred = X_b @ theta

    # Preventing NaN/Inf so code doesn't crash
    if np.any(np.isnan(y_pred)) or np.any(np.isinf(y_pred)):
        print(model_name)
        print("  slope:", theta[1])
        print("  MSE: diverged (NaN/Inf)")
        print("  R^2: diverged (NaN/Inf)")
        print()
        continue

    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    print(model_name)
    print("  intercept:", theta[0])
    print("  slope:", theta[1])
    print("  MSE:", mse)
    print("  R^2:", r2)
    print()

Linear (lambda=0)
  intercept: 1.1377269735725186
  slope: 1.94527518078825
  MSE: 1.9499357635589158
  R^2: 0.725823805647487

Ridge (lambda=1)
  intercept: 1.1376714374282075
  slope: 1.943850219320922
  MSE: 1.9499385334705766
  R^2: 0.7258234161762789

Ridge (lambda=10)
  intercept: 1.1371752494913991
  slope: 1.9311188945363944
  MSE: 1.95020913827749
  R^2: 0.7257853670274199

Ridge (lambda=100)
  intercept: 1.1325488744360113
  slope: 1.8124141074269553
  MSE: 1.9740156919277065
  R^2: 0.7224379796916689

Ridge (lambda=1000)
  intercept: 1.1056067050737797
  slope: 1.1224487965504333
  MSE: 2.873519116993794
  R^2: 0.5959607743905316

Ridge (lambda=10000)
  intercept: 1.0709647236308166
  slope: 0.23350905283446305
  MSE: 5.947068166910269
  R^2: 0.16379577828633074



When lambda = 0 (ordinary linear regression), the estimated slope is close to the true value 2, and the model achieves the lowest MSE and highest R-squared.For small values of lambda (1 and 10), the slope changes only slightly, and the performance metrics remain almost the same, indicating that mild regularization has little effect. However, as lambda increases to 100 and 100, the slope shrinks toward zero, which is caused by ridge regression. The MSE increases and R-squared decreases, which means the model underfits the data. When lambda is 10,000, the slope is close to zero, meaning the model ignores the relationship between X and Y. The MSE becomes larger and R-squared is negative, showing strong underfitting. In conclusion, as lambda increases, the model starts to underfit and coefficients shrink to zero, meaning smaller lambda helps with good performance.